In [1]:
!pip install deepface tensorflow-gpu==2.10

In [11]:
import cv2
import os
from deepface import DeepFace
from tqdm import tqdm
import json
import random


def extract_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    idx = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break  # If no more frames, exit the loop

        frames.append((idx, frame))
        idx += 1
    
    cap.release()
    frames = random.sample(frames, min(len(frames), 50))
    return frames


def get_scene_gender(video_path):
    scene = os.path.splitext(os.path.basename(video_path))[0]
    scene_dict = {"scene": scene}
    genres_list = []
    frames = extract_frames(video_path)
    for (idx, frame) in (frames):  # Analyzing first 10 frames
        frame_list = []
        result = DeepFace.analyze(frame, actions=['gender'], enforce_detection=False)

        if result:
            for i, res in enumerate(result):
                frame_list.append(
                    {
                        'id': f'{scene}_{idx}_{i}',
                        'frame': idx,
                        'F': res['gender']['Woman'], 
                        'M': res['gender']['Man'],
                        'x': res['region']['x'],
                        'y': res['region']['y'],
                        'width': res['region']['w'],
                        'height': res['region']['h']
                    }
                )
        else:
            frame_list.append({'id': f'{scene}_{idx}', 'frame': idx})
        
        genres_list.append(frame_list)
    
    scene_dict['genres'] = genres_list
    return scene_dict

def get_movie_gender(folder_path, film_code, output_path):
    scene_dict = {}
    scene_dict['film'] = film_code
    for scene in tqdm(os.listdir(f'{folder_path}/{film_code}')):
        if not scene.endswith('.txt'):
            path = f'{folder_path}/{film_code}/{scene}'
            scene_dict[scene] = get_scene_gender(path)
    with open(f'{output_path}/{film_code}.json', 'w') as out:
        json.dump(scene_dict, out, indent=4)



def main():
    folder_path = '../scenes'
    output_path = '../genre'
    get_movie_gender(folder_path, 'tt0822832', output_path)


main()


100%|█████████████████████████████████████████████████████████████████████████████████| 86/86 [1:00:17<00:00, 42.06s/it]
